# Executing DAG Workflows with Parsl-RP
In this tutorial, we will execute a simple DAG (Directed Acyclic Graph) with Parsl-RP, which involves leveraging the Parsl data flow management and the Radical Pilot (RP) workload management to facilitate the concurrent execution of tasks.

<div style="text-align:center"><img src="https://airflow.apache.org/docs/apache-airflow/stable/_images/basic-dag.png" alt="my image" width="30%"></div>}

In [ ]:
# make sure Parsl is installed
!pip show parsl

In [ ]:
# make sure RCT tools are installed and up to date
! radical-stack

First, let's import Parsl and RP Python modules in our application, alongside the RadicalPilotExecutor (RPEX) from Parsl

In [ ]:
import parsl
from parsl.config import Config
from parsl.app.app import python_app, bash_app
from parsl.executors.radical import ResourceConfig
from parsl.executors.radical import RadicalPilotExecutor

Now, let's declare the `Resource Config`, which helps the RCT runtime system to determine how many resources are required for RAPTOR engine. In our case, we do not specify any and let RCT use the default (4 cores)

In [ ]:
rpex_cfg = ResourceConfig()

Now, let's create a Parsl Config which helps Parsl to identiy which executor to run the tasks on.

In [ ]:
config = Config(executors=[RadicalPilotExecutor(
                           label='RPEXBulk',
                           rpex_cfg=rpex_cfg,
                           resource='local.localhost',
                           runtime=30, cores=8)])

radical_executor = config.executors[0]

Now, let's load the the config via Parsl

In [ ]:
parsl.load(config)

In [ ]:
@python_app
def task_a(a):
    import random
    b = random.randint(1, 10)
    c = random.randint(1, 10)
    return b, c

@python_app
def task_b(b, parsl_resource_specification={}):
    return b * 2

@python_app
def task_c(c, parsl_resource_specification={}):

    return c * 2

# Reduce function that returns the sum of a list
@python_app
def task_d(inputs):
    x = sum(inputs)
    return x

# task_a will run first to generate two numbers and pass them to
# task_b and task_c
t1 = task_a(100)
b, c = t1.result()
print('Task-A is Finished')


# t2 and t3 will run conccurently
t2 = task_b(b, parsl_resource_specification={'ranks':1})
t3 = task_c(c, parsl_resource_specification={'ranks':1})

# t4 will run after t2 and t3 finishes
t4 = task_d([t2.result(), t3.result()])
print('Task-B is Finished', 'Task-C is Finished')

# Now, let's print the results of our final task
print('Task-D result:', t4.result())

Finally, shutdown the executor, otherwise it will always stays ready to get more tasks

In [ ]:
radical_executor.shutdown()